# Preparación y exploración de datos

Una vez conocidas las estructuras de datos de pandas, las operaciones básicas que se pueden realizar sobre las mismas y el modo en el que realizar la carga y almacenamiento de dichas estructuras en discos, vamos a centrarnos en aquellas funcionalidades ofrecidas por pandas que están más orientadas al tratamiento y análisis de datos.

In [3]:
import numpy as np
import pandas as pd

## Gestión de datos en blanco (<i>missing values</i>)

En la mayoría de los ficheros utilizados como fuente de datos, es muy común la existencia de valores nulos (en blanco, <i>missing</i>...). Estos "huecos" en la información suelen ser muy problemáticos ya que tiene un impacto importante a la hora de realizar cualquier tipo de cálculo numérico y son difícilmente interpretables.<br/>
Uno de los objetivos de pandas en su construcción fue facilitar el tratamiento de este tipo de datos no existentes ofreciendo múltiples funciones que permiten llevar a cabo tanto su detección, como su eliminación o imputación...

#### Detección de <i>missing values</i>

Pandas ofrece principalmente dos funciones para manejar la detección de valores nulos.<br/>
<ul>
<li><b>isnull:</b> Que devuelve una Serie o DataFrame booleano indicando qué elemetos son NaN o None.</li>
<li><b>notnull:</b> Que devuelve el inverso del anterior.</li>

In [4]:
catastro = pd.read_table('datos/catastro.tsv', nrows=10)
catastro

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,NaN,NaN,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,NaN,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,NaN,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,NaN,1.142542e+08


In [3]:
# Detección de valores nulos
catastro.isnull()

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,True,False
5,False,False,False,False,False,False,False,False,False,False,True,False
6,False,False,False,False,False,False,False,False,True,True,False,False
7,False,False,False,False,False,False,False,False,False,False,True,False
8,False,False,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,False,True,False


In [5]:
catastro.shape

(10, 12)

In [5]:
len(catastro) # Cuanrtas filas tiene

10

In [6]:
len(catastro.columns) # Cuantas columnas tiene

12

In [4]:
# Detección de valores no nulos
catastro.notnull()

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,True,True,True,True,True,True,True,True,True,True,False,True
1,True,True,True,True,True,True,True,True,True,True,False,True
2,True,True,True,True,True,True,True,True,True,True,False,True
3,True,True,True,True,True,True,True,True,True,True,False,True
4,True,True,True,True,True,True,True,True,True,True,False,True
5,True,True,True,True,True,True,True,True,True,True,False,True
6,True,True,True,True,True,True,True,True,False,False,True,True
7,True,True,True,True,True,True,True,True,True,True,False,True
8,True,True,True,True,True,True,True,True,True,True,False,True
9,True,True,True,True,True,True,True,True,True,True,False,True


In [7]:
catastro.isnull().sum() # Sumo cuantos NaN hay

año                0
id_distrito        0
distrito           0
id_barrio          0
barrio             0
id_uso             0
uso                0
num_inmuebles      0
año_cons_medio     1
sup_cons           1
sup_suelo          9
valor_catastral    0
dtype: int64

#### Eliminación de registros con <i>missing values</i>

Aunque SIEMPRE conviene hacer un estudio cuidadoso del por qué y la casuística de los valores nulos, uno de los posibles tratamientos a aplicar es su eliminación directa del set de datos. Pandas, nos ofrece el método <b>dropna</b> para llevar a cabo esta tarea. Los parámetros de este método son:<br/>
<ul>
<li><b>axis:</b> Selección de eje sobre el que realizar la eliminación.</li>
<li><b>how:</b> Tomará posibles valores 'any' y 'all' e indica si se debe eliminar la fila o columna cuando haya uno o más valores NaN o cuando todos los valores sean NaN.</li>
<li><b>thresh:</b> Permite indicar, el número de observaciones no nulas que se deben tener para no realizar el borrado.</li>
</ul>

In [8]:
catastro = pd.read_table('datos/catastro.tsv', nrows=10)
catastro

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,NaN,NaN,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,NaN,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,NaN,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,NaN,1.142542e+08


In [9]:
# Eliminación de filas con al menos 1 NA (any)
catastro.dropna(axis=0, how='any')

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral


In [10]:
# Eliminación de columnas con al menos 1 NA
catastro.dropna(axis=1, how='any')

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1.142542e+08


In [11]:
# Eliminación de filas con 2 o más NA
catastro.dropna(thresh=len(catastro.columns)-1)

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,NaN,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,NaN,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,NaN,1.142542e+08


#### Imputación de registros con <i>missing values</i>

Existirán casos en los que no se desee (o no se pueda) eliminar los registros con valores nulos (p.e. podrían suponer un porcentaje demasiado elevado de nuestro set de datos). En estos casos, habrá que realizar una imputación de los mismos a un valor preestablecidor.<br/>
Pandas pone a nuestra disposición el método <b>fillna</b>, que cuenta, entre otros, con los siguientes parámetros:<br/>
<ul>
<li><b>axis:</b> Que decide si aplicará el criterio de relleno por filas o columnas.</li>
<li><b>value:</b> Que rellena los valores nulos a un valor fijo.</li>
</ul>

In [12]:
catastro = pd.read_table('datos/catastro.tsv', nrows=10)
catastro

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,NaN,NaN,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,NaN,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,NaN,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,NaN,1.142542e+08


In [13]:
# Imputación de valores a 0. Todos los NaN se convierten en 0
catastro.fillna(0)

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,0.0,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,0.0,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,0.0,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,0.0,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,0.0,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,0.0,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,0.0,0.0,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,0.0,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,0.0,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,0.0,1.142542e+08


In [14]:
# Imputación de valores por valor anterior (por columnas)
catastro.fillna(method='ffill')
# Propaga los valores apenas consiga uno y los esparce hacia abajo, si antes no consiguio uno, dejará los NaN

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,1946.0,7238.0,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,130010.0,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,130010.0,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,130010.0,1.142542e+08


In [15]:
# Imputación de valores por valor anterior (por columnas)
catastro.fillna(method='bfill', limit=3)
# Se esparce a lo contrario; con limit le restrinjo que solo debe realizarlo 3 veces

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,1946.0,7238.0,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,130010.0,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,130010.0,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,130010.0,1.142542e+08


## Resumen de datos y estadísticos básicos

Al igual que NumPy, pandas ofrece un conjunto amplio de funciones para llevar a cabo un análisis estadístico de datos.  Las más relevantes serían:<br/>
<ul>
<li><b>describe:</b> Presenta un conjunto con las estadísticas básicas más comunes calculadas sobre todas las columnas de la estructura. Equivalente a la función <i>summary</i> de R.</li>
<li><b>count:</b> Número de elementos no nulos.</li>
<li><b>min, max:</b> Valor mínimo y máximo.</li>
<li><b>argmin, argmax, idxmax, idxmin:</b> Posiciones con valor mínimo y máximo.</li>
<li><b>quantile:</b> Cuantil calculado.</li>
<li><b>sum:</b> Suma de elementos.</li>
<li><b>mean:</b> Media aritmética de los elementos.</li>
<li><b>median:</b> Mediana de los elementos.</li>
<li><b>std:</b> Desviación estándar de los elementos.</li>
<li><b>var:</b> Varianza de los elementos.</li>
<li><b>cumsum:</b> Suma acumulada de los elementos.</li>
<li><b>cumprod:</b> Producto acumulado de los elementos.</li>
</ul>

La mayor parte de estos métodos, podrán recibir 3 parámetros:
<ul>
<li><b>axis:</b> Que indica si realizar el cálculo por filas o columnas.</li>
<li><b>skipna:</b> Que indica si se deben ignorar o no los valores NaN a la hora de realizar los cálculos.</li>
</ul>

In [21]:
catastro = pd.read_table('datos/catastro.tsv', nrows=10)
catastro

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
5,2014,1,Centro,11,PALACIO,K,Deportivo,8,1946.0,7238.0,NaN,1.061466e+07
6,2014,1,Centro,11,PALACIO,M,"Suelos sin edificar, obras de urbanización y j...",47,NaN,NaN,130010.0,1.991520e+07
7,2014,1,Centro,11,PALACIO,O,Oficinas,559,1947.0,196893.0,NaN,3.407841e+08
8,2014,1,Centro,11,PALACIO,P,Edificio Singular,15,1891.0,197518.0,NaN,2.816668e+08
9,2014,1,Centro,11,PALACIO,R,Religioso,17,1884.0,102718.0,NaN,1.142542e+08


In [17]:
# Estadísticos básicos sobre el data set
catastro.describe()

,año,id_distrito,id_barrio,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
count,10.0,10.0,10.0,10.000000,9.000000,9.000000,1.0,1.000000e+01
mean,2014.0,1.0,11.0,539.900000,1928.444444,125865.888889,130010.0,1.587417e+08
std,0.0,0.0,0.0,980.444848,27.559532,85847.096668,NaN,1.429993e+08
min,2014.0,1.0,11.0,8.000000,1884.000000,7238.000000,130010.0,1.061466e+07
25%,2014.0,1.0,11.0,18.250000,1919.000000,62963.000000,130010.0,3.389358e+07
50%,2014.0,1.0,11.0,41.500000,1937.000000,114226.000000,130010.0,1.218900e+08
75%,2014.0,1.0,11.0,482.750000,1946.000000,197518.000000,130010.0,2.601035e+08
max,2014.0,1.0,11.0,3034.000000,1969.000000,223552.000000,130010.0,4.076055e+08


In [18]:
# Suma por columnas
catastro.sum()

año                                                            20140
id_distrito                                                       10
distrito           CentroCentroCentroCentroCentroCentroCentroCent...
id_barrio                                                        110
barrio             PALACIOPALACIOPALACIOPALACIOPALACIOPALACIOPALA...
id_uso                                                    ACEGIKMOPR
uso                Almacén-EstacionamientoComercialCulturalOcio y...
num_inmuebles                                                   5399
año_cons_medio                                               17356.0
sup_cons                                                   1132793.0
sup_suelo                                                   130010.0
valor_catastral                                        1587416814.45
dtype: object

In [19]:
# Suma por filas ignorando NA
catastro.sum(axis=1, skipna=True)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [22]:
catastro.mean()

TypeError: Could not convert ['CentroCentroCentroCentroCentroCentroCentroCentroCentroCentro'
 'PALACIOPALACIOPALACIOPALACIOPALACIOPALACIOPALACIOPALACIOPALACIOPALACIO'
 'ACEGIKMOPR'
 'Almacén-EstacionamientoComercialCulturalOcio y HosteleríaIndustrialDeportivoSuelos sin edificar, obras de urbanización y jardineríaOficinasEdificio SingularReligioso'] to numeric

## Elementos únicos y frecuencias

<ul>
<li><b>unique</b>: Que nos devuelve un array con el conjunto de elementos únicos de una Serie.</li>
<li><b>value_counts</b>: Que realiza un cálculo de frecuencias sobre los elementos únicos de una Serie.</li>
<li><b>isin:</b> Que nos permite chequear si un conjunto de valores se encuentra en una Serie.</li>
</ul>

In [23]:
catastro = pd.read_table('datos/catastro.tsv')
catastro

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
3025,2013,21,Barajas,215,CORRALEJOS,O,Oficinas,172,1994.0,538345.0,NaN,6.893051e+08
3026,2013,21,Barajas,215,CORRALEJOS,R,Religioso,2,2006.0,3264.0,NaN,3.059075e+06
3027,2013,21,Barajas,215,CORRALEJOS,T,Espectáculos,1,1995.0,14720.0,NaN,1.639676e+07
3028,2013,21,Barajas,215,CORRALEJOS,V,Residencial,2939,1999.0,433231.0,NaN,5.177294e+08


In [24]:
# Conjunto de de barrios sin repetir
catastro.barrio.unique()

array(['PALACIO', 'EMBAJADORES', 'CORTES', 'JUSTICIA', 'UNIVERSIDAD',
       'SOL', 'IMPERIAL', 'ACACIAS', 'CHOPERA', 'LEGAZPI', 'DELICIAS',
       'PALOS DE MOGUER', 'ATOCHA', 'PACÍFICO', 'ADELFAS', 'ESTRELLA',
       'IBIZA', 'LOS JERÓNIMOS', 'NIÑO JESÚS', 'RECOLETOS', 'GOYA',
       'FUENTE DEL BERRO', 'GUINDALERA', 'LISTA', 'CASTELLANA', 'EL VISO',
       'PROSPERIDAD', 'CIUDAD JARDÍN', 'HISPANOAMÉRICA', 'NUEVA ESPAÑA',
       'CASTILLA', 'BELLAS VISTAS', 'CUATRO CAMINOS', 'CASTILLEJOS',
       'ALMENARA', 'VALDEACEDERAS', 'BERRUGUETE', 'GAZTAMBIDE',
       'ARAPILES', 'TRAFALGAR', 'ALMAGRO', 'RIOS ROSAS', 'VALLEHERMOSO',
       'EL PARDO', 'FUENTELARREINA', 'PEÑA GRANDE', 'EL PILAR', 'LA PAZ',
       'VALVERDE', 'MIRASIERRA', 'EL GOLOSO', 'CASA DE CAMPO',
       'ARGÜELLES', 'CIUDAD UNIVERSITARIA', 'VALDEZARZA', 'VALDEMARÍN',
       'EL PLANTÍO', 'ARAVACA', 'LOS CARMENES', 'PUERTA DEL ANGEL',
       'LUCERO', 'ALUCHE', 'CAMPAMENTO', 'CUATRO VIENTOS', 'LAS AGUILAS',
       'COMILLA

In [25]:
# Tabla de frecuencias de distritos osea, cuantas veces aparece cierto distro en el dataframe
catastro.distrito.value_counts()

distrito
Ciudad Lineal            214
Fuencarral - El Pardo    189
San Blas                 186
Moncloa - Aravaca        172
Carabanchel              166
Latina                   165
Usera                    160
Centro                   156
Salamanca                155
Arganzuela               153
Puente de Vallecas       150
Chamartín                148
Chamberí                 147
Tetuán                   144
Hortaleza                142
Retiro                   137
Moratalaz                125
Barajas                  114
Villaverde               112
Vicálvaro                 49
Villa de Vallecas         46
Name: count, dtype: int64

In [27]:
catastro['distrito'].isin(['Centro'])

0        True
1        True
2        True
3        True
4        True
        ...  
3025    False
3026    False
3027    False
3028    False
3029    False
Name: distrito, Length: 3030, dtype: bool

In [26]:
# Chequeo de existencia de distritos
catastro[catastro['distrito'].isin(['Centro', 'Latina'])]

,año,id_distrito,distrito,id_barrio,barrio,id_uso,uso,num_inmuebles,año_cons_medio,sup_cons,sup_suelo,valor_catastral
0,2014,1,Centro,11,PALACIO,A,Almacén-Estacionamiento,3034,1969.0,214457.0,NaN,1.295259e+08
1,2014,1,Centro,11,PALACIO,C,Comercial,1407,1921.0,223552.0,NaN,4.076055e+08
2,2014,1,Centro,11,PALACIO,E,Cultural,36,1937.0,62963.0,NaN,7.582872e+07
3,2014,1,Centro,11,PALACIO,G,Ocio y Hostelería,254,1919.0,114226.0,NaN,1.954138e+08
4,2014,1,Centro,11,PALACIO,I,Industrial,22,1942.0,13228.0,NaN,1.180795e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
2294,2013,10,Latina,107,LAS AGUILAS,O,Oficinas,80,1994.0,12218.0,NaN,1.413729e+07
2295,2013,10,Latina,107,LAS AGUILAS,P,Edificio Singular,7,1966.0,59258.0,NaN,3.585992e+07
2296,2013,10,Latina,107,LAS AGUILAS,R,Religioso,6,1985.0,7135.0,NaN,5.193782e+06
2297,2013,10,Latina,107,LAS AGUILAS,V,Residencial,22925,1973.0,1902523.0,NaN,1.398108e+09


## Aplicación de funciones sobre estructuras

Al igual que en R tenemos la familia de funciones <i>apply</i>, pandas pone a nuestra disposición un conjunto de funciones que nos permiten aplicar operaciones elemento a elemento (o fila a fila, o columna a columna) en sus estructuras de datos. En concreto disponemos de tres funciones.

#### Aplicación de funciones elemento a elemento sobre Series - Función map

In [28]:
serie = pd.Series([1, 2, 3, 4, 5, 6])
serie

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64

In [29]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)
es_par(3)

'Impar: 3'

In [30]:
# Aplicación de función elemento a elemento sobre Serie
serie.map(es_par)

0    Impar: 1
1      Par: 2
2    Impar: 3
3      Par: 4
4    Impar: 5
5      Par: 6
dtype: object

#### Aplicación de funciones elemento a elemento sobre DataFrames - Función applymap

In [31]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4))
dataframe

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [32]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)

In [33]:
# Aplicación de función elemento a elemento sobre DataFrame
dataframe.applymap(es_par)

,0,1,2,3
0,Par: 0,Impar: 1,Par: 2,Impar: 3
1,Par: 4,Impar: 5,Par: 6,Impar: 7
2,Par: 8,Impar: 9,Par: 10,Impar: 11
3,Par: 12,Impar: 13,Par: 14,Impar: 15


#### Aplicación de funciones fila a fila o columna a columna sobre DataFrames - Función apply

In [34]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4))
dataframe

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [35]:
def es_suma_par(elemento):
    if np.sum(elemento) % 2 == 0:
        return 'Suma par: ' + str(np.sum(elemento))
    else:
        return 'Suma impar: ' + str(np.sum(elemento))

In [38]:
# Aplicación de función por columnas sobre DataFrame
dataframe.apply(es_suma_par)#, axis=0)

0    Suma par: 24
1    Suma par: 28
2    Suma par: 32
3    Suma par: 36
dtype: object

In [39]:
# Aplicación de función por filas sobre DataFrames
dataframe.apply(es_suma_par, axis=1)

0     Suma par: 6
1    Suma par: 22
2    Suma par: 38
3    Suma par: 54
dtype: object

## Fusión de estructuras

La librería pandas nos ofrece, principalmente, dos formas de fusionar estructuras de datos: realizando cruces entre ellos (mediante las claves coincidentes de sus índices) o concatenando sus contenidos (bien por filas o columnas).

#### Función merge - JOIN de estructuras

In [40]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013], 
             'Valoración':[6, None, 8.75, None],
             'Presupuesto':[160, 250, 100, None],
             'Director':['Gareth Edwards','Peter Jackson',  'Martin Scorsese', 'Alfonso Cuarón'],
             'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']}
)
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Gareth Edwards,Godzilla
1,2014,NaN,250.0,Peter Jackson,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity


In [41]:
directores = pd.DataFrame(
            {'Director':['Gareth Edwards', 'Martin Scorsese', 'Pedro Almodovar'],
             'AñoNacimiento':[1975, 1942, 1949],
             'Nacionalidad': ['England', 'USA', 'Spain']
             }
)
directores

,Director,AñoNacimiento,Nacionalidad
0,Gareth Edwards,1975,England
1,Martin Scorsese,1942,USA
2,Pedro Almodovar,1949,Spain


In [42]:
pd.merge(peliculas, directores[['Director', 'AñoNacimiento']], on = ['Director'])

,Año,Valoración,Presupuesto,Director,Título,AñoNacimiento
0,2014,6.00,160.0,Gareth Edwards,Godzilla,1975
1,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street,1942


La función busca, por defecto, aquellas claves de columnas que coinciden y realiza el cruce, eliminando del resultado aquellas filas para las que el cruce no es posible.<br/>

También podemos especificar, explícitamente, el conjunto de columnas a utilizar en el cruce.

In [44]:
directores.columns = ['Nombre', 'Nacimiento', 'Nacionalidad']
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre')

,Año,Valoración,Presupuesto,Director,Título,Nombre,Nacimiento,Nacionalidad
0,2014,6.00,160.0,Gareth Edwards,Godzilla,Gareth Edwards,1975,England
1,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street,Martin Scorsese,1942,USA


Por último, al igual que ocurre en os JOIN de SQL, podemos especificar el modo de cruce a aplicar, haciendo que las filas de la estructura de la izquierda, derecha o ambas que no coincidan se mantengan en el resultado, estableciendo valores NaN en aquellos elementos para los que no exista información.

In [45]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='left')


,Año,Valoración,Presupuesto,Director,Título,Nombre,Nacimiento,Nacionalidad
0,2014,6.00,160.0,Gareth Edwards,Godzilla,Gareth Edwards,1975.0,England
1,2014,NaN,250.0,Peter Jackson,El Hobbit III,NaN,NaN,NaN
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street,Martin Scorsese,1942.0,USA
3,2013,NaN,NaN,Alfonso Cuarón,Gravity,NaN,NaN,NaN


In [46]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='right')

,Año,Valoración,Presupuesto,Director,Título,Nombre,Nacimiento,Nacionalidad
0,2014.0,6.00,160.0,Gareth Edwards,Godzilla,Gareth Edwards,1975,England
1,2013.0,8.75,100.0,Martin Scorsese,El lobo de Wall Street,Martin Scorsese,1942,USA
2,NaN,NaN,NaN,NaN,NaN,Pedro Almodovar,1949,Spain


In [47]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='outer')

,Año,Valoración,Presupuesto,Director,Título,Nombre,Nacimiento,Nacionalidad
0,2014.0,6.00,160.0,Gareth Edwards,Godzilla,Gareth Edwards,1975.0,England
1,2014.0,NaN,250.0,Peter Jackson,El Hobbit III,NaN,NaN,NaN
2,2013.0,8.75,100.0,Martin Scorsese,El lobo de Wall Street,Martin Scorsese,1942.0,USA
3,2013.0,NaN,NaN,Alfonso Cuarón,Gravity,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Pedro Almodovar,1949.0,Spain


Finalmente, en el caso de que tengamos columnas duplicadas en los dos DataFrames que se van a unir, pandas se encargará automáticamente de incluir un sufijo que permita desambiguar (_x, _y, por defecto). 

In [48]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013], 
             'Valoración':[6, None, 8.75, None],
             'Presupuesto':[160, 250, 100, None],
             'Director':['Peter Jackson', 'Gareth Edwards', 'Martin Scorsese', 'Alfonso Cuarón'],
             'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']}
)
directores = pd.DataFrame(
            {'Director':['Gareth Edwards', 'Martin Scorsese', 'Pedro Almodovar'],
             'AñoNacimiento':[1975, 1942, 1949],
             'Nacionalidad': ['England', 'USA', 'Spain'],
             'Valoración':[6, 7, 8]
             }
)
pd.merge(peliculas, directores, left_on='Director', right_on='Director')

,Año,Valoración_x,Presupuesto,Director,Título,AñoNacimiento,Nacionalidad,Valoración_y
0,2014,NaN,250.0,Gareth Edwards,El Hobbit III,1975,England,6
1,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street,1942,USA,7


Si queremos modificar estos sufijos, podemos hacer uso del parámetro suffixes que recibe una tupla con los sufijos a utilizar.

In [49]:
pd.merge(peliculas, directores, left_on='Director', right_on='Director', suffixes=('_peli', '_dire'))

,Año,Valoración_peli,Presupuesto,Director,Título,AñoNacimiento,Nacionalidad,Valoración_dire
0,2014,NaN,250.0,Gareth Edwards,El Hobbit III,1975,England,6
1,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street,1942,USA,7


#### Función concat

Esta función nos permite fusionar estructuras sin realizar ningún tipo de cruce entre ellas, sino "colocándolas" juntas para la creación de una estructura mayor. Podemos hacerlo tanto en filas como en columnas. Por defecto, se concatenan filas y se mantienen las columnas de ambas estructuras aunque no coincidan en clave, dejando a NaN los elementos que no existan.

In [50]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013], 
             'Valoración':[6, None, 8.75, None],
             'Presupuesto':[160, 250, 100, None],
             'Director':['Gareth Edwards','Peter Jackson', 'Martin Scorsese', 'Alfonso Cuarón']},
            index = ['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']
)
peliculas2 = pd.DataFrame(
            {'Año':[2014, 2014], 
             'Valoración':[7.3, 6.3],
             'Director':['Evan Goldberg', ' Rupert Wyatt']},
            index = ['La entrevista', 'El jugador']
)
peliculas


,Año,Valoración,Presupuesto,Director
Godzilla,2014,6.00,160.0,Gareth Edwards
El Hobbit III,2014,NaN,250.0,Peter Jackson
El lobo de Wall Street,2013,8.75,100.0,Martin Scorsese
Gravity,2013,NaN,NaN,Alfonso Cuarón


In [51]:
peliculas2

,Año,Valoración,Director
La entrevista,2014,7.3,Evan Goldberg
El jugador,2014,6.3,Rupert Wyatt


In [52]:
pd.concat([peliculas, peliculas2], sort = False)

,Año,Valoración,Presupuesto,Director
Godzilla,2014,6.00,160.0,Gareth Edwards
El Hobbit III,2014,NaN,250.0,Peter Jackson
El lobo de Wall Street,2013,8.75,100.0,Martin Scorsese
Gravity,2013,NaN,NaN,Alfonso Cuarón
La entrevista,2014,7.30,NaN,Evan Goldberg
El jugador,2014,6.30,NaN,Rupert Wyatt


También podemos concatenar por columnas.

In [53]:
peliculas3 = pd.DataFrame(
            {'Recaudación':[525, 722, 392]},
            index = ['Godzilla', 'El Hobbit III', 'El lobo de Wall Street']
)
# Queria concatenar una nueva columna que estaba en un nuevo DataFrame, esto lo hago con axis=1
pd.concat([peliculas, peliculas3],axis=1, sort = False)

,Año,Valoración,Presupuesto,Director,Recaudación
Godzilla,2014,6.00,160.0,Gareth Edwards,525.0
El Hobbit III,2014,NaN,250.0,Peter Jackson,722.0
El lobo de Wall Street,2013,8.75,100.0,Martin Scorsese,392.0
Gravity,2013,NaN,NaN,Alfonso Cuarón,NaN


Aunque es un funcionamiento más propio de la función merge, pandas nos permite eliminar del resultado aquellas combinaciones para las que no existen datos en alguna de las dos estructuras.

In [54]:
pd.concat([peliculas, peliculas3],axis=1, join='inner')

,Año,Valoración,Presupuesto,Director,Recaudación
Godzilla,2014,6.00,160.0,Gareth Edwards,525
El Hobbit III,2014,NaN,250.0,Peter Jackson,722
El lobo de Wall Street,2013,8.75,100.0,Martin Scorsese,392


Por último, puede ser útil identificar en la estructura resultante el origen de cada una de las filas para posterior análisis. La función concat incluye un parámetro <b>keys</b> que podemos utilizar para añadir una clave a cada uno de las estructuras origen, que se convertirá en el nivel más agregado de un índice jerárquico.

In [55]:
pd.concat([peliculas, peliculas2], keys=['dataset1','dataset2'], sort = False)

Año  Valoración  Presupuesto  \
dataset1 Godzilla                2014        6.00        160.0   
         El Hobbit III           2014         NaN        250.0   
         El lobo de Wall Street  2013        8.75        100.0   
         Gravity                 2013         NaN          NaN   
dataset2 La entrevista           2014        7.30          NaN   
         El jugador              2014        6.30          NaN   

                                        Director  
dataset1 Godzilla                 Gareth Edwards  
         El Hobbit III             Peter Jackson  
         El lobo de Wall Street  Martin Scorsese  
         Gravity                  Alfonso Cuarón  
dataset2 La entrevista             Evan Goldberg  
         El jugador                 Rupert Wyatt

## Operaciones de agrupación

La librería pandas también incluye la posibiilidad de hacer agrupación de resultados y operaciones sobre los grupos (al estilo de las sentencias GROUP BY de SQL).

In [56]:
peliculas

,Año,Valoración,Presupuesto,Director
Godzilla,2014,6.00,160.0,Gareth Edwards
El Hobbit III,2014,NaN,250.0,Peter Jackson
El lobo de Wall Street,2013,8.75,100.0,Martin Scorsese
Gravity,2013,NaN,NaN,Alfonso Cuarón


In [57]:
agrupado = peliculas.groupby('Año')
type(agrupado)

pandas.core.groupby.generic.DataFrameGroupBy

Una agrupación no es un objeto "imprimible", es una representación interna del conjunto de registros que pertenecen a cada grupo y sólo tiene sentido si, posteriormente, se va a aplicar alguna operación sobre dichos grupos. Hay que tener en cuenta que no todas las operaciones son aplicacables sobre todos los tipos de columna.

In [58]:
# Media por grupo
agrupado.mean()

TypeError: Could not convert Martin ScorseseAlfonso Cuarón to numeric

In [59]:
# Conteo de valores no nulos por grupo
agrupado.count()

,Valoración,Presupuesto,Director
Año,,,
2013,1,1,2
2014,1,2,2


Aunque con estos datos quizá no tenga tanto sentido, podemos realizar la agrupación por múltiples claves.

In [60]:
peliculas.groupby(['Año', 'Director']).sum()

Valoración  Presupuesto
Año  Director                                
2013 Alfonso Cuarón         0.00          0.0
     Martin Scorsese        8.75        100.0
2014 Gareth Edwards         6.00        160.0
     Peter Jackson          0.00        250.0